In [1]:
import os
import csv
import copy
import math

import pandas as pd

from itertools import combinations

C:\Users\amitg\AppData\Local\Temp\ipykernel_16840\455751299.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
graphs_dir = "graphs"
results_dir = "results"

In [3]:
graph_name = "graph"

graph = {}
with open(os.path.join(graphs_dir, f"{graph_name}.txt"), "r") as f:
    line = f.readline()
    while line:
        node_edges = line.split()
        node = node_edges[0]        
        edges = node_edges[1:]
        graph[node] = set(edges)
        line = f.readline()

graph

{'A': {'B', 'C', 'D'}, 'B': {'A'}, 'C': {'A'}, 'D': {'A'}}

In [4]:
nodes = list(graph.keys())
node_positions = {v: i for i, v in enumerate(nodes)}

In [5]:
# max_degree = max(len(graph[n]) for n in nodes)
# degree_of_nodes = {n: max_degree for n in nodes}

degree_of_nodes = {n: len(graph[n]) for n in nodes}

print("Degree of all nodes (starting from 0):")
degree_of_nodes # start from 0

Degree of all nodes (starting from 0):


{'A': 3, 'B': 1, 'C': 1, 'D': 1}

In [6]:
configurations = {
    tuple([0 for i in range(len(nodes))])
}
# perturb each state at a time for all states in configurations and accumulate the same in the configurations for next state to perturb
for n in nodes:
    node_pos = node_positions[n]
    config_copy = copy.deepcopy(configurations)
    for i in range(1, degree_of_nodes[n]+1):
        for cc in config_copy:
            cc = list(cc)
            cc[node_pos] = i
            configurations.add(tuple(cc))

print("No. of Configurations:", len(configurations))

No. of Configurations: 32


In [7]:
invariants = set()
for state in configurations:
    all_paths = combinations(range(len(state)), 2)
    for src, dest in all_paths:
        src_node, dest_node = nodes[src], nodes[dest]
        src_color, dest_color = state[src], state[dest]
        if dest_node in graph[src_node] and src_color == dest_color:
            # found same color node between neighbors
            break
    else:
        invariants.add(state)

print("Invariants and Count of Invariants:")
len(invariants)

Invariants and Count of Invariants:


18

In [8]:
program_transitions_rank = {}
for inv in invariants:
    program_transitions_rank[inv] = {"L": 0, "C": 1, "A": 0, "Ar": 0, "M": 0}

In [9]:
def find_min_possible_color(colors):
    for i in range(len(colors)+1):
        if i not in colors:
            return i

In [10]:
def is_different_color(color, other_colors):
    """
    return True if "color" is different from all "other_colors"
    """
    for c in other_colors:
        if color == c:
            return False
    return True

In [11]:
def is_program_transition(perturb_pos, start_state, dest_state):
    if start_state in invariants and dest_state in invariants:
        return False

    node = nodes[perturb_pos]
    neighbor_pos = [node_positions[n] for n in graph[node]]
    neighbor_colors = set(dest_state[i] for i in neighbor_pos)
    min_color = find_min_possible_color(neighbor_colors)
    return dest_state[perturb_pos] == min_color

In [12]:
def get_program_transitions(start_state):
    program_transitions = set()
    for position, val in enumerate(start_state):
        # check if node already has different color among the neighbors => If yes => no need to perturb that node's value
        node = nodes[position]
        neighbor_pos = [node_positions[n] for n in graph[node]]
        neighbor_colors = set(start_state[i] for i in neighbor_pos)
        if is_different_color(val, neighbor_colors):
            continue
        
        # if the current node's color is not different among the neighbors => search for the program transitions possible
        possible_node_colors = set(range(degree_of_nodes[nodes[position]]+1))
        for perturb_val in possible_node_colors:
            perturb_state = list(start_state)
            perturb_state[position] = perturb_val
            perturb_state = tuple(perturb_state)
            if perturb_state != start_state:
                if is_program_transition(position, start_state, perturb_state):
                    program_transitions.add(perturb_state)
    return {"program_transitions": program_transitions}

In [13]:
def get_cvfs(start_state):
    cvfs_in = dict()
    cvfs_out = dict()
    for position, _ in enumerate(start_state):
        possible_node_colors = set(range(degree_of_nodes[nodes[position]]+1))
        for perturb_val in possible_node_colors:
            perturb_state = list(start_state)
            perturb_state[position] = perturb_val
            perturb_state = tuple(perturb_state)
            if perturb_state != start_state:
                if start_state in invariants:
                    cvfs_in[perturb_state] = position # track the nodes to calculate its overall rank effect
                else:
                    cvfs_out[perturb_state] = position
    return {"cvfs_in": cvfs_in, "cvfs_out": cvfs_out}

In [14]:
program_transitions_n_cvf = {}

for state in configurations:
    program_transitions_n_cvf[state] = {**get_program_transitions(state), **get_cvfs(state)}

In [15]:
unranked_states = set(program_transitions_n_cvf.keys()) - set(program_transitions_rank.keys())
print("Unranked states for Program transitions:", len(unranked_states))

Unranked states for Program transitions: 14


In [16]:
# rank the states that has all the paths to the ranked one
while unranked_states:
    ranked_states = set(program_transitions_rank.keys())
    remove_from_unranked_states = set()
    for state in unranked_states:
        dests = program_transitions_n_cvf[state]['program_transitions']
        if dests - ranked_states:       # some desitnations states are yet to be ranked
            pass
        else:                           # all the destination has been ranked
            total_path_length = 0
            path_count = 0
            _max = 0
            for succ in dests:
                path_count += program_transitions_rank[succ]["C"]
                total_path_length += program_transitions_rank[succ]["L"] + program_transitions_rank[succ]["C"]
                _max = max(_max, program_transitions_rank[succ]["M"])
            program_transitions_rank[state] = {
                "L": total_path_length,
                "C": path_count,
                "A": total_path_length/path_count,
                "Ar": math.ceil(total_path_length/path_count),
                "M": _max + 1
            }
            remove_from_unranked_states.add(state)
    unranked_states -= remove_from_unranked_states

In [17]:
program_transitions_rank

{(3, 0, 1, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (3, 1, 1, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 1, 1, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 1, 1, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 0, 0, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (3, 1, 1, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (3, 0, 0, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 0, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (3, 0, 1, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (3, 1, 0, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 0, 1, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 1, 0, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 1, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (3, 0, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (3, 1, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (1, 0, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 1, 1, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 0, 1, 0):